读着读着发现，事情不是我想象的那么简单。先mark一下，随着以后的进度回过头来再解决这个问题。

In [ ]:
class StochasticGreedy(BaseOptimizer):
    
    # 代码来源：斯坦福大佬
    # https://github.com/jmschrei/apricot
    
    # 参数注释：
    # https://apricot-select.readthedocs.io/en/latest/optimizers/stochastic.html  

    def __init__(self, function=None, epsilon=0.9, random_state=None, 
        n_jobs=None, verbose=False):
        self.epsilon = epsilon
        super(StochasticGreedy, self).__init__(function=function, 
            random_state=random_state, n_jobs=n_jobs, verbose=verbose)
        
    '''
        function 就是需要次模优化的函数。不过好像至今还没有碰到logdet
        epsilon和抽样的子集合R的大小有关，也和最后的总计算量有关。见论文。
    '''
    
    def select(self, X, k, sample_cost=None):
        cost = 0.0
        if sample_cost is None:
            sample_cost = numpy.ones(X.shape[0], dtype='float64')

        n = X.shape[0]
        subset_size = -numpy.log(self.epsilon) * n / k
        subset_size = max(int(subset_size), 1)
    
    '''
        subset_size就是子集的规模，套用论文里的公式。
    '''
        
        
        while cost < k:
            idxs = self.random_state.choice(self.function.idxs, 
                replace=False, size=min(subset_size, 
                    self.function.idxs.shape[0]))

            gains = self.function._calculate_gains(X, idxs) / sample_cost[idxs]
            idxs_ = numpy.lexsort((numpy.arange(gains.shape[0]), -gains))
    '''
        多级排序：https://blog.csdn.net/Flag_ing/article/details/124185378
        numpy抽样：https://blog.csdn.net/qq_30868235/article/details/80409227
    '''
            for idx in idxs_:
                best_idx = idxs[idx]
                if cost + sample_cost[best_idx] <= k:
                    break
                else:
                    return

            cost += sample_cost[best_idx]
            self.function._select_next(X[best_idx], gains[idx], best_idx)

            if self.verbose == True:
                self.function.pbar.update(1)